# Replica del entrenamiento del modelo text2midi

En este notebook se replica todo el proceso de entrenamiento presentado en el paper [Text2midi: Generating Symbolic Music from Captions](http://arxiv.org/abs/2412.16526).

El modelo se puede encontrar perfectamente en hugging face en el [siguiente enlace](https://huggingface.co/amaai-lab/text2midi)

## Preparación del dataset SymphonyNet

Primero se realiza el preprocesamiento del dataset usado para el pre entrenamiento. Se utiliza la librería music 21 para extraer los atributos de tempo, tonalidad, bpm e instrumentos de los archivos midi. Estos atributos sirven como placeholder de 10 frases diferentes, para simplificar usaremos una única plantilla. 

### Prueba de music 21

Primero se prueba que music 21 lea correctamente uno de los archivos midi antes de crear pseudo captions.

In [ ]:
# Importar librería music21
from music21 import converter, tempo, key
import os

# Definir la ruta del archivo MIDI
midi_path = "../datasets/SymphonyNet_Dataset/classical/altnikol_befiehl_du_deine_wege_(c)icking-archive.mid"

# Verificar que el archivo existe
if os.path.exists(midi_path):
    print(f"Archivo encontrado: {midi_path}")
else:
    print(f"Archivo NO encontrado: {midi_path}")
    
# Cargar el archivo MIDI
score = converter.parse(midi_path)
print("\nArchivo MIDI cargado correctamente")
print(f"Tipo de objeto: {type(score)}")

Archivo encontrado: ../datasets/SymphonyNet_Dataset/classical/altnikol_befiehl_du_deine_wege_(c)icking-archive.mid

Archivo MIDI cargado correctamente
Tipo de objeto: <class 'music21.stream.base.Score'>

Archivo MIDI cargado correctamente
Tipo de objeto: <class 'music21.stream.base.Score'>


Ahora que sabemos que el archivo se leyó correctamente. Extraemos los atributos requeridos para el pre entrenamiento.

In [18]:
# Mostrar información básica del score
print("=" * 60)
print("INFORMACIÓN BÁSICA DEL ARCHIVO MIDI")
print("=" * 60)
print(f"\nNúmero de partes (tracks): {len(score.parts)}")
print(f"Duración total: {score.quarterLength} quarter notes")
print(f"Duración en segundos: {score.seconds:.2f} segundos")

INFORMACIÓN BÁSICA DEL ARCHIVO MIDI

Número de partes (tracks): 4
Duración total: 1878.0 quarter notes
Duración en segundos: nan segundos


In [23]:
# Crear un resumen estructurado de los atributos extraídos
print("=" * 60)
print("RESUMEN DE ATRIBUTOS EXTRAÍDOS")
print("=" * 60)

# Función auxiliar para extraer atributos de forma robusta
def extract_musical_attributes(score):
    attributes = {
        'tempo_bpm': None,
        'key': None,
        'key_mode': None,
        'time_signature': None,
        'instruments': set(),
        'num_tracks': len(score.parts),
        'duration_seconds': round(score.seconds, 2),
        'num_notes': len(score.flatten().notes)
    }
    
    # Tempo
    tempo_marks = score.flatten().getElementsByClass(tempo.MetronomeMark)
    if tempo_marks:
        attributes['tempo_bpm'] = tempo_marks[0].number
    else:
        try:
            estimated_tempo = score.flatten().metronomeMarkBoundaries()[0][-1]
            attributes['tempo_bpm'] = estimated_tempo.number
        except:
            pass
    
    # Tonalidad
    key_sigs = score.flatten().getElementsByClass(key.Key)
    if key_sigs:
        attributes['key'] = key_sigs[0].tonic.name
        attributes['key_mode'] = key_sigs[0].mode
    else:
        try:
            analyzed_key = score.analyze('key')
            attributes['key'] = analyzed_key.tonic.name
            attributes['key_mode'] = analyzed_key.mode
        except:
            pass
    
    # Compás
    time_sigs = score.flatten().getElementsByClass('TimeSignature')
    if time_sigs:
        attributes['time_signature'] = f"{time_sigs[0].numerator}/{time_sigs[0].denominator}"
    
    # Instrumentos
    for instrument in score.getInstruments() :
        if instrument:
            attributes['instruments'].add(instrument.bestName())

    attributes['instruments'] = ', '.join(attributes['instruments']) if attributes['instruments'] else None
    
    return attributes

# Extraer y mostrar atributos
attributes = extract_musical_attributes(score)

print("\nAtributos extraídos:")
for k, value in attributes.items():
    print(f"  {k}: {value}")

print("\n" + "=" * 60)

RESUMEN DE ATRIBUTOS EXTRAÍDOS

Atributos extraídos:
  tempo_bpm: 120
  key: D
  key_mode: minor
  time_signature: 4/4
  instruments: Pan Flute, Trombone, English Horn, Bass, Alt, Tenor, Violoncello, Sopran
  num_tracks: 4
  duration_seconds: nan
  num_notes: 8139



In [25]:
PSEUDO_TEMPLATE = f"A musical piece in {{key}} {{key_mode}} key with a tempo of {{tempo_bpm}} BPM, time signature of {{time_signature}}, featuring instruments: {{instruments}}."

In [26]:
print(PSEUDO_TEMPLATE.format(**attributes))

A musical piece in D minor key with a tempo of 120 BPM, time signature of 4/4, featuring instruments: Pan Flute, Trombone, English Horn, Bass, Alt, Tenor, Violoncello, Sopran.


Vemos que usando *music21* se obtiene los atributos musicales necesarios para generar las pseudo captions. Esta función se usara más adelante tras iterar por todos los archivos y generar un dataframe con los atributos **location** y **caption** para mantener los nombres del MidiCaps dataset.

### Generar el Dataframe para SymphonyNetDataset

En esta sección se genera el dataframe que sera usado como pre entrenamiento del modelo 